In [254]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [255]:
#创建C1
def createC1(dataSet):
    C1=[]
    for transaction in dataSet:
        for goods in transaction:
            if [goods] not in C1:
                C1.append([goods])
    C1.sort()
    C1=map(frozenset,C1)
    return C1

In [256]:
dataSet=loadDataSet()
dataSet=map(set,dataSet)
C1=createC1(dataSet)

In [257]:
#计算支持度,选择Lk
def createLk(dataSet,Ck,minSup=0.5):
    retList=[]
    supData={}
    retDic={}
    numItem=len(dataSet)
    for Ci in Ck:
        numSub=0
        for transaction in dataSet:
            if Ci.issubset(transaction):
                numSub+=1
        if (float(numSub)/numItem)>=minSup:
            retList.append(Ci)
            supData[Ci]=(float(numSub)/numItem)
    return retList,supData

In [258]:
createLk(dataSet,C1,0.5)

([frozenset({1}), frozenset({2}), frozenset({3}), frozenset({5})],
 {frozenset({5}): 0.75,
  frozenset({2}): 0.75,
  frozenset({3}): 0.75,
  frozenset({1}): 0.5})

In [279]:
#完整的Apriori算法
def Apriori(dataSet,minSup=0.5):
    C1=createC1(dataSet)
    L=[];supData={}
    L1,supData1=createLk(dataSet,C1,minSup)
    L.append(L1);supData.update(supData1)
    C2=[]
    for i in range(len(L1)-1):
        for j in range(i+1,len(L1)):
            C2.append(L1[i]|L1[j])
    L2,supData2=createLk(dataSet,C2,minSup)
    L.append(L2);supData.update(supData2)
    Ltemp=L2[:]
    #当K从3开始
    k=3
    while(True):
        Ctemp=[]
        for i in range(len(Ltemp)-1):
            for j in range(i+1,len(Ltemp)):
                li1=(list(Ltemp[i])[:k-2])
                li2=(list(Ltemp[j])[:k-2])
                li1.sort();li2.sort()
                if li1==li2:
                    Ctemp.append(Ltemp[i]|Ltemp[j])
        if len(Ctemp)==0:
            break
        Ltemp,supDatatemp=createLk(dataSet,Ctemp,minSup)
        L.append(Ltemp);supData.update(supDatatemp)
        k+=1
        if len(Ltemp)==1:
            break
    return L,supData

In [264]:
L,supData=Apriori(dataSet,0.5)

In [265]:
#计算单项链接的置信度
def calConfi(itemList,subsets,supData,minConfi):
    retList=[]
    condiList=[]
    for item in itemList:
        confidence=supData[subsets]/supData[item]
        if confidence>minConfi:
            retList.append((item,subsets-item,confidence))
            condiList.append(item)
    return retList,condiList

#创建关联规则
def generateRules(L,supData,minConfi=0.7):
    retList=[]
    #第二层的情况
    for subsets in L[1]:
        itemList=[frozenset([item]) for item in list(subsets)]
        retList.extend(calConfi(itemList,subsets,supData,minConfi)[0])
    #第三层之后
    for lyr in range(2,len(L)):
        for subsets in L[lyr]:
            #第1层（1项）
            itemList=[frozenset([item]) for item in list(subsets)]
            retList1,condiList1=calConfi(itemList,subsets,supData,minConfi)
            retList.extend(retList1)
            #第2层（2项）
            C2=[]
            for i in range(len(condiList1)-1):
                for j in range(i+1,len(condiList1)):
                    C2.append(condiList1[i]|condiList1[j])
            itemList=[frozenset([item]) for item in list(C2)]
            retList1,condiList1=calConfi(C2,subsets,supData,minConfi)
            retList.extend(retList1)
            #当K从3开始
            k=3
            while(True):
                Ctemp=[]
                for i in range(len(condiList1)-1):
                    for j in range(i+1,len(condiList1)):
                        li1=(list(condiList1[i])[:k-2])
                        li2=(list(condiList1[j])[:k-2])
                        li1.sort();li2.sort()
                        if li1==li2:
                            Ctemp.append(condiList1[i]|condiList1[j])
                if len(Ctemp)<=1:
                    break
                retList1,condiList1=calConfi(Ctemp,subsets,supData,minConfi)
                retList.extend(retList1)
                k+=1
    return retList

In [266]:
generateRules(L,supData)

[(frozenset({1}), frozenset({3}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0)]

In [280]:
#采用数据测试
with open('mushroom.dat','r') as fread:
    lines=fread.readlines()
    mushDataSet=[]
    for line in lines:
        mushDataSet.append(line.split())
L,supData=Apriori(mushDataSet,0.3)
for item in L[1]:
    if item.intersection('2'):print item

frozenset(['2', '23'])
frozenset(['2', '28'])
frozenset(['2', '34'])
frozenset(['2', '36'])
frozenset(['39', '2'])
frozenset(['2', '53'])
frozenset(['2', '59'])
frozenset(['63', '2'])
frozenset(['2', '67'])
frozenset(['76', '2'])
frozenset(['2', '85'])
frozenset(['2', '86'])
frozenset(['90', '2'])
frozenset(['93', '2'])
